### SettingWithCopyWarning の例 (pandas2)  
- https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
- https://pandas.pydata.org/pandas-docs/stable/user_guide/copy_on_write.html#copy-on-write
- ポイント: データフレームから部分を取り出し、loc以外への代入でそれを変更すると、元のデータフレームも同時に変更される場合と変更されない場合がある。
- そのような危険があるときに SettingWithCopyWarningが発生するが、確実ではなく、実際に元のデータフレームが変更されるかどうかは状況次第でユーザからは実際のところわからない。
- locを使うと、確実にlocに付随するオブジェクトだけが変更される。
- データフレームから部分を取り出す際にcopy()を使うと、元のデータフレームとは別の実体が生成されるため、変更しても元のデータフレームは変更されず、問題は起きない。
- pandas3 からは SettingWithCopyWarning は発生しない (Copy-on-Write, CoW が必ずON)。
  - CoW: 取り出された部分が参照されるだけならコピーされず、変更されるときに自動的に別の実体が生成される(メモリ利用効率や実行速度が向上)

#### pandas2 であることを確認  

In [1]:
import pandas as pd
print(pd.__version__)

2.2.3


#### データフレームdfを作成  

In [2]:
lst = [[0,1,2,3],[4,5,6,7],[8,9,10,11]]
df = pd.DataFrame(lst, columns=['c0','c1','c2','c3'])
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


#### dfからc1, c3列を取り出してds2に代入  

In [3]:
df2 = df[['c1', 'c3']]
display(df2)

,c1,c3
0,1,3
1,5,7
2,9,11


#### df2のc3列を、loc以外への代入で上書き変更 (SettingWithCopyWarning発生)  

In [4]:
df2['c3'] = df2['c3']*10
display(df2)

C:\Users\shugo\AppData\Local\Temp\ipykernel_5784\7379380.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['c3'] = df2['c3']*10


,c1,c3
0,1,30
1,5,70
2,9,110


#### dfは変化していない (変なことは起きていない) が、たまたまかもしれない    

In [5]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


#### df2のc1列を取り出してserに代入  

In [6]:
ser = df2['c1']
print(ser)

0    1
1    5
2    9
Name: c1, dtype: int64


#### serの1つの要素を値変更 (SettingWithCopyWarning発生)    

In [7]:
ser[1] = 55
print(ser)

0     1
1    55
2     9
Name: c1, dtype: int64


C:\Users\shugo\AppData\Local\Temp\ipykernel_5784\300457834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ser[1] = 55


#### df2も同時に変更されている  

In [8]:
display(df2)

,c1,c3
0,1,30
1,55,70
2,9,110


#### dfは変更されていない  

In [9]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


#### 今度はdf2ではなくdfからc1列を取り出してser2に代入  

In [10]:
ser2 = df['c1']
print(ser2)

0    1
1    5
2    9
Name: c1, dtype: int64


#### ser2の1つの要素を値変更 (SettingWithCopyWarningは発生しない)  

In [11]:
ser2[1] = 555
print(ser2)

0      1
1    555
2      9
Name: c1, dtype: int64


#### dfも同時に変更されている！  

In [12]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,555,6,7
2,8,9,10,11


#### to_numpy() でdfからndarray部分を取り出してndに代入  

In [13]:
nd = df.to_numpy()
print(nd)

[[  0   1   2   3]
 [  4 555   6   7]
 [  8   9  10  11]]


#### ndの1つの値を変更  

In [14]:
nd[1, 2] = 666
print(nd)

[[  0   1   2   3]
 [  4 555 666   7]
 [  8   9  10  11]]


#### dfも同時に変更されている  

In [15]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,555,666,7
2,8,9,10,11


#### あらためて、loc以外/locへの代入の比較  

In [16]:
lst = [[0,1,2,3],[4,5,6,7],[8,9,10,11]]
df = pd.DataFrame(lst, columns=['c0','c1','c2','c3'])
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


##### loc以外に代入 (SettingWithCopyWarning発生)    

In [17]:
df2 = df[['c1', 'c3']]
df2['c1'] *= 10
display(df2)

C:\Users\shugo\AppData\Local\Temp\ipykernel_5784\2933704738.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['c1'] *= 10


,c1,c3
0,10,3
1,50,7
2,90,11


##### dfは変化していない (たまたまかもしれない)  

In [18]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


##### locに代入 (SettingWithCopyWarningは発生しない)        

In [19]:
df3 = df[['c1', 'c3']]
df3.loc[:, 'c1'] *= 10
display(df3)

,c1,c3
0,10,3
1,50,7
2,90,11


##### dfは変化しない (確実)    

In [20]:
display(df)

,c0,c1,c2,c3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
